In [1]:
from transformers import pipeline
import torch
from lang_sam import LangSAM
import numpy as np
from segment_anything import SamPredictor, sam_model_registry
print(torch.cuda.is_available())
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline, save_pretrained
from transformers import T5ForConditionalGeneration, T5Tokenizer

/Users/sudhanshu/Desktop/UMASS_COURSES_SEMESTERS/SEM_2/NLP/Project/vqa/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


False


In [27]:
model_name = "google/flan-t5-base"
class NERModel:
    def __init__(self, model_path="google/flan-t5-base"):
        self.ner = T5ForConditionalGeneration.from_pretrained(model_path)
        self.tokenizer = T5Tokenizer.from_pretrained(model_path)

    
    def extract_entities(self, instruction, text):
        # Formulate the prompt according to FLAN T5 expected format
        prompt = f"{instruction}: {text}"
        
        # Encode the prompt
        input_ids = self.tokenizer.encode(prompt, return_tensors="pt", max_length=512, truncation=True)
        
        # Generate the output from the model
        outputs = self.ner.generate(input_ids, max_length=512, num_return_sequences=1)
        
        # Decode the generated ids to text
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response
    def save_model(self, save_path):
        self.ner.save_pretrained(save_path)
        self.tokenizer.save_pretrained(save_path)
    
class SamModelPrediction:
    def __init__(self):
        
        self.sam = LangSAM("vit_h","/Users/sudhanshu/Desktop/UMASS_COURSES_SEMESTERS/SEM_2/NLP/Project/modelCheckpoint/sam_vit_h_4b8939.pth")
        #self.predictor = SamPredictor(self.sam)

    def predict_image(self, image, prompt):
        self.mask, _, _, _ = self.sam.predict(image, prompt)
        return self.mask
        # self.predictor.set_image(image)  
        # input_box = np.array([100, 100, 400, 400])
        # self.masks, _, _ = self.predictor.predict(prompt) 
        # return self.masks 




In [28]:
ner = NERModel()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [29]:

instruction = "Extract common nouns and persons from the following sentences:"
prompt = """
1. Sentence: "The cat sat on the mat?" Common Nouns: cat, sat, mat.
2. Sentence: "He drinks a lot of coffee and reads many books." Common Nouns:  coffee, reads, books.
3. Sentence: "Sky is blue and grass is green." Common Nouns: sky, blue, grass, green.
4. Sentence: "Is the  Boy is skateboarding on the wall?" Common Nouns: boy, skateboarding, wall .
5. Sentence: "What color is teddy bear " Common Nouns: teddy,bear, color
Sentence: "Fat man sitting on the couch."
Common Nouns:"""

# Encode the prompt
response = ner.extract_entities(instruction, prompt)

print(response)

fat man, sitting


In [32]:
model_path = 'lang-segment-anything/fineTunedT5ForNER'
ner.save_model('./fineTunedT5ForNER')
